# The Attention Mechanism from Scratch

The attention mechanism was introduced to improve the performance of the encoder-decoder model for machine translation. The idea behind the attention mechanism was to permit the decoder to utilize the most relevant parts of the input sequence in a flexible manner, by a weighted combination of all the encoded input vectors, with the most relevant vectors being attributed the highest weights. 

## The Attention Mechanism

The attention mechanism was introduced by [Bahdanau et al. (2014)](https://arxiv.org/abs/1409.0473) to address the bottleneck problem that arises with the use of a fixed-length encoding vector, where the decoder would have limited access to the information provided by the input. This is thought to become especially problematic for long and/or complex sequences, where the dimensionality of their representation would be forced to be the same as for shorter or simpler sequences.

Note that Bahdanau et al.’s attention mechanism is divided into the step-by-step computations of the <b>alignment scores</b>, <b>the weights</b>, and the <b>context vector</b>:

- 1. <b>Alignment scores</b>: The alignment model takes the encoded hidden states, h<sub>i</sub>, and the previous decoder output, S<sub>t-1</sub>, to compute a score, e<sub>t, i</sub>, that indicates how well the elements of the input sequence align with the current output at the position, t. The alignment model is represented by a function, a(.) , which can be implemented by a feedforward neural network:
$$e_{t,i}=a(S_{t-1}, h_i)$$

- 2. <b>Weights</b>: The weights, $\alpha_{t,i}$, are computed by applying a softmax operation to the previously computed alignment scores:
$$\alpha_{t,i} = softmax(e_{t,i})$$

- 3. <b>Context vector</b>: A unique context vector, C<sub>t</sub>, is fed into the decoder at each time step. It is computed by a weighted sum of all, T, encoder hidden states:
$$C_t = \sum_{i=1}^{T}\alpha_{t,i}h_i$$

However, the attention mechanism can be re-formulated into a general form that can be applied to any sequence-to-sequence (abbreviated to seq2seq) task, where the information may not necessarily be related in a sequential fashion. 

## The General Attention Mechanism

The general attention mechanism makes use of three main components, namely the queries, Q, the keys, K, and the values, V. 

If you had to compare these three components to the attention mechanism as proposed by Bahdanau et al., then the query would be analogous to the previous decoder output, S<sub>t-1</sub>, while the values would be analogous to the encoded inputs, h<sub>i</sub>. 

The general attention mechanism then performs the following computations:

- 1. Each query vector, q = S<sub>t-1</sub>, is matched against a database of keys to compute a score value. This matching operation is computed as the dot product of the specific query under consideration with each key vector, k<sub>i</sub>: 
$$e_{q,k_i}=q  ^.  k_i$$

- 2. The scores are passed through a softmax operation to generate the weights:
$$\alpha_{q,k_i} = softmax(e_{q,k_i})$$

- 3. The generalized attention is then computed by a weighted sum of the value vectors, V<sub>ki</sub>, where each value vector is paired with a corresponding key:
$$attention(q, K, V) = \sum_i\alpha_{q,k_i}V_{k_i}$$

Within the context of machine translation, each word in an input sentence would be attributed its own query, key, and value vectors. These vectors are generated by multiplying the encoder’s representation of the specific word under consideration with three different weight matrices that would have been generated during training. 

In essence, when the generalized attention mechanism is presented with a sequence of words, it takes the query vector attributed to some specific word in the sequence and scores it against each key in the database. In doing so, it captures how the word under consideration relates to the others in the sequence. Then it scales the values according to the attention weights (computed from the scores) to retain focus on those words relevant to the query. In doing so, it produces an attention output for the word under consideration. 

## The General Attention Mechanism with NumPy and SciPy

In [1]:
import numpy as np
import scipy

Let’s start by first defining the word embeddings of the four different words to calculate the attention. In actual practice, these word embeddings would have been generated by an encoder; however, for this particular example, you will define them manually. 

In [5]:
# encoder representations of four different words
word_1 = np.array([1, 0, 0])
word_2 = np.array([0, 1, 0])
word_3 = np.array([1, 1, 0])
word_4 = np.array([0, 0, 1])

# stacking the word embeddings into a single array
words = np.array([word_1, word_2, word_3, word_4])

The next step generates the weight matrices, which you will eventually multiply to the word embeddings to generate the queries, keys, and values. Here, you shall generate these weight matrices randomly; however, in actual practice, these would have been learned during training. 

In [7]:
# generating the weight matrices
np.random.seed(42) # to allow us to reproduce the same attention values
W_Q = np.random.randint(3, size=(3, 3))
W_K = np.random.randint(3, size=(3, 3))
W_V = np.random.randint(3, size=(3, 3))

Notice how the number of rows of each of these matrices is equal to the dimensionality of the word embeddings (which in this case is three) to allow us to perform the matrix multiplication.

Subsequently, the query, key, and value vectors for each word are generated by multiplying each word embedding by each of the weight matrices. 

In [8]:
# generating the queries, keys and values
query_1 = word_1 @ W_Q
key_1 = word_1 @ W_K
value_1 = word_1 @ W_V
 
query_2 = word_2 @ W_Q
key_2 = word_2 @ W_K
value_2 = word_2 @ W_V
 
query_3 = word_3 @ W_Q
key_3 = word_3 @ W_K
value_3 = word_3 @ W_V
 
query_4 = word_4 @ W_Q
key_4 = word_4 @ W_K
value_4 = word_4 @ W_V

Considering only the first word for the time being, the next step scores its query vector against all the key vectors using a dot product operation. 

In [12]:
# scoring the first query vector against all key vectors
scores = np.array([np.dot(query_1, key_1), np.dot(query_1, key_2), np.dot(query_1, key_3), np.dot(query_1, key_4)])
print(scores)

[ 8  2 10  2]


The score values are subsequently passed through a softmax operation to generate the weights. Before doing so, it is common practice to divide the score values by the square root of the dimensionality of the key vectors (in this case, three) to keep the gradients stable. 

In [14]:
# computing the weights by a softmax operation
weights = scipy.special.softmax(scores / key_1.shape[0] ** 0.5)
print(weights)

[0.23608986 0.00738988 0.74913039 0.00738988]


Finally, the attention output is calculated by a weighted sum of all four value vectors. 

In [15]:
# computing the attention by a weighted sum of the value vectors
attention = (weights[0] * value_1) + (weights[1] * value_2) + (weights[2] * value_3) + (weights[3] * value_4)
 
print(attention)

[0.98522025 1.74174051 0.75652026]


For faster processing, the same calculations can be implemented in matrix form to generate an attention output for all four words in one go:

In [16]:
# generating the queries, keys and values
Q = words @ W_Q
K = words @ W_K
V = words @ W_V
 
# scoring the query vectors against all key vectors
scores = Q @ K.transpose()
 
# computing the weights by a softmax operation
weights = scipy.special.softmax(scores / K.shape[1] ** 0.5, axis=1)
 
# computing the attention by a weighted sum of the value vectors
attention = weights @ V
 
print(attention)


[[0.98522025 1.74174051 0.75652026]
 [0.90965265 1.40965265 0.5       ]
 [0.99851226 1.75849334 0.75998108]
 [0.99560386 1.90407309 0.90846923]]
